In [ ]:
%pip install -qU langchain langchain-core langchain-community langchain-openai langgraph python-dotenv

In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain

In [3]:
load_dotenv()

openai_api_key = os.environ["OPENAI_API_KEY"]
model_name = "gpt-3.5-turbo-0125"
llm = ChatOpenAI(
    model=model_name,
    openai_api_key=openai_api_key,
    temperature=0.1,
    max_tokens=1000
)

In [4]:
db_file_path = "./chinook.db"
db = SQLDatabase.from_uri(f"sqlite:///{db_file_path}")

In [ ]:
print(db.dialect)
print(db.get_usable_table_names())

In [ ]:
query = "SELECT * FROM Artist LIMIT 10;"

db.run(query)

In [8]:
chain = create_sql_query_chain(llm, db)

In [ ]:
response = chain.invoke({
    "question": "How many employees are there in the organization?"
})

print(response)

In [ ]:
db.run(response)

In [11]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.tools import tool


@tool
def parse(query_string: str) -> str:
    """
    Parses Generated SQL Statement
    """

    splitted = query_string.split(":")

    if (len(splitted) >= 2):
        query = splitted[1]
    else:
        query = query_string

    return query

In [ ]:
# GPT 4 Generated SQL Statement

query = """
QUERY: SELECT COUNT("EmployeeId") AS "TotalEmployees"
FROM "Employee"
"""

print(parse.invoke(query))

# GPT 3.5 Generated SQL Statement
query = """
SELECT COUNT("EmployeeId") AS "TotalEmployees"
FROM "Employee"
"""

print(parse.invoke(query))

In [17]:
execute_query = QuerySQLDataBaseTool(db = db)
write_query = create_sql_query_chain(llm, db)

chain = write_query | parse | execute_query

In [ ]:
question = "How many employees are there in the organization?"

response = chain.invoke({
    "question": question
})

print(response)

In [19]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

answer_prompt = PromptTemplate.from_template(
    """
    Given the following user question, corresponding SQL Query, and SQL Result, answer the user question.

    Question: {question}
    SQL Query: {query}
    Result: {result}
    Answer:
    """
)

chain = RunnablePassthrough.assign(query = write_query) \
    .assign(result = itemgetter("query") | parse | execute_query) | \
    answer_prompt | llm | StrOutputParser()

In [ ]:
question = "How many employees are there in the organization?"

response = chain.invoke({
    "question": question
})

print(response)

In [ ]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db = db, llm = llm)

tools = toolkit.get_tools()

tools

In [23]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage, SystemMessage

In [ ]:
SQL_PREFIX = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables.
"""

system_message = SystemMessage(content=SQL_PREFIX)
agent_executor = create_react_agent(
    llm, tools, messages_modifier=system_message
)

In [ ]:
for stream in agent_executor.stream({
    "messages": [
        HumanMessage(content="which country's customers have spent the most?")
    ]
}):
    print(stream)
    print("******")

In [ ]:
for stream in agent_executor.stream({
    "messages": [
        HumanMessage(content="Describe the playlisttrack table.")
    ]
}):
    print(stream)
    print("******")

In [ ]:
for stream in agent_executor.stream({
    "messages": [
        HumanMessage(content="Top 5 customers who have spent most buying MPEG Audio, and heavy metal type of playlists.")
    ]
}):
    print(stream)
    print("******")